### Farm Planning ###

In [1]:
from pyomo.environ import *

# Crear modelo
model = ConcreteModel()

# Conjuntos
model.Years = RangeSet(1, 5)
model.Lands = RangeSet(1, 4)
model.Ages = RangeSet(1, 12)

# Parámetros
model.housing_cap = Param(initialize=130)
model.land_cap = Param(initialize=200)
model.labor_cap = Param(initialize=5500)
model.installment = Param(initialize=39.71)

model.grain_intake = Param(initialize=0.6)
model.sugar_intake = Param(initialize=0.7)

model.heifer_decay = Param(initialize=0.05)
model.cow_decay = Param(initialize=0.02)
model.birthrate = Param(initialize=1.1)

model.milk_price = Param(initialize=370)
model.bl_price = Param(initialize=30)
model.hf_price = Param(initialize=40)
model.cow_price = Param(initialize=120)

grain_yield = {1: 1.10, 2: 0.90, 3: 0.80, 4: 0.65}
grain_area = {1: 20, 2: 30, 3: 20, 4: 10}
model.gr_yield = Param(model.Lands, initialize=grain_yield)
model.gr_area = Param(model.Lands, initialize=grain_area)

# Variables de decisión
model.Outlay = Var(model.Years, domain=NonNegativeReals)
model.Overtime = Var(model.Years, domain=NonNegativeReals)
model.Newborn = Var(model.Years, domain=NonNegativeReals)
model.HeiferSell = Var(model.Years, domain=NonNegativeReals)
model.Profit = Var(model.Years, domain=Reals)
model.Cows = Var(model.Years, model.Ages, domain=NonNegativeReals)
model.GR = Var(model.Years, model.Lands, domain=NonNegativeReals)
model.SB = Var(model.Years, domain=NonNegativeReals)

# Función objetivo
def objective_rule(m):
    return sum(m.Profit[t] - m.installment * (t + 4) * m.Outlay[t] for t in m.Years)
model.Objective = Objective(rule=objective_rule, sense=maximize)

# Restricción de capacidad de alojamiento
def housing_capacity_rule(m, t):
    return m.Newborn[t] + sum(m.Cows[t, k] for k in m.Ages if k != 12) <=            m.housing_cap + sum(m.Outlay[d] for d in m.Years if d <= t)
model.HousingConstraint = Constraint(model.Years, rule=housing_capacity_rule)

# Resolver el modelo
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

# Mostrar resultados
model.Profit.display()

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\AMINAD~1\AppData\Local\Temp\tmptnhmnxrb.glpk.raw --wglp
 C:\Users\AMINAD~1\AppData\Local\Temp\tmpwpr64b56.glpk.glp --cpxlp C:\Users\AMINAD~1\AppData\Local\Temp\tmpk5y6ppxz.pyomo.lp
Reading problem data from 'C:\Users\AMINAD~1\AppData\Local\Temp\tmpk5y6ppxz.pyomo.lp'...
5 rows, 70 columns, 75 non-zeros
179 lines were read
Writing problem data to 'C:\Users\AMINAD~1\AppData\Local\Temp\tmpwpr64b56.glpk.glp'...
173 lines were written
GLPK Simplex Optimizer, v4.65
5 rows, 70 columns, 75 non-zeros
Preprocessing...
PROBLEM HAS NO DUAL FEASIBLE SOLUTION
If you need actual output for non-optimal solution, use --nopresol
Time used:   0.0 secs
Memory used: 0.0 Mb (40045 bytes)
Writing basic solution to 'C:\Users\AMINAD~1\AppData\Local\Temp\tmptnhmnxrb.glpk.raw'...
84 lines were written
Profit : Size=5, Index=Years
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :  None :  None :  None :